## Data Handling

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
np.random.seed(4444)

In [36]:
df = pd.read_csv('../../../../../jaeyeun/01_nh_poc/15_split_data_set_and_make_json_for_train_test_set/split_70_15_15.csv')

df.head()

,file_name,label,raw_text,dataset_n,train_val_test
0,R1509261.txt,0,보 도 자 료\nhttp://www.msip.go.kr 보도일시 2015. 9. 4...,0,0
1,R2003733.txt,0,보도일시 2020. 3. 18.(수) 조간(온라인 3. 17. 12:00)부터 보도...,0,0
2,D1507076-1.txt,0,보 도 자 료\nhttp://www.msip.go.kr 보도일시 2015. 7. 1...,0,0
3,R2005031.txt,0,보 도 자 료\n배포일시 2020. 4. 29.(수) 총 4매(본문2) 담당 부서 ...,0,0
4,R2006226.txt,0,<전매체> 2020년 6월 3일(수) 10:00(국무회의 개최시)부터 보도하여 주시...,0,0


* Train : KDI 70+15 (train+val)
* Val : KDI 15(test)
* Test : legal 273(train+val+test)

In [37]:
X_train_df = df[(df['dataset_n'] == 0) & ((df['train_val_test'] == 0) | (df['train_val_test'] == 1))]
X_val_df = df[(df['dataset_n'] == 0) & (df['train_val_test'] == 2)]
X_test_df = df[df['dataset_n'] != 0]
y_train = df[(df['dataset_n'] == 0) & ((df['train_val_test'] == 0) | (df['train_val_test'] == 1))]['label']
y_val = df[(df['dataset_n'] == 0) & (df['train_val_test'] == 2)]['label']
y_test = df[df['dataset_n'] != 0]['label']

In [38]:
print(X_train_df.shape)
print(X_val_df.shape)
print(X_test_df.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(9174, 5)
(1626, 5)
(273, 5)
(9174,)
(1626,)
(273,)


In [39]:
n_features = 100000
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.9, # 0.9 만큼의 문서 이상에서 나오면 거른다.
    min_df=5, # 5개 미만의 문서에서 나오면 거른다.
    sublinear_tf = True, # tf value를 완만하게 처리 (outlier 처리 효과)
    ngram_range = (1, 3),
    max_features=n_features)

In [40]:
X_train_tfidf = tfidf_vectorizer.fit(X_train_df['raw_text'])
# X_test_tfidf = tfidf_vectorizer.fit_transform(prep_text_test)
# X_test_hash = hash_vectorizer.fit_transform(prep_text_test)

In [41]:
X_train = X_train_tfidf.transform(X_train_df['raw_text'])
X_val = X_train_tfidf.transform(X_val_df['raw_text'])
X_test = X_train_tfidf.transform(X_test_df['raw_text'])

In [42]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(9174, 100000)
(1626, 100000)
(273, 100000)


In [43]:
svd = TruncatedSVD(n_components=100)
svd.fit(X_train_tfidf_transformed)

TruncatedSVD(n_components=100)

In [44]:
X_train = svd.transform(X_train_tfidf_transformed)
X_val = svd.transform(X_val_tfidf_transformed)
X_test = svd.transform(X_test_tfidf_transformed)

In [45]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(9174, 100)
(1626, 100)
(273, 100)


## Classify

* LightGBM RandomSearch

In [46]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgbm

In [47]:
lgbm_clf = lgbm.LGBMClassifier()

lgbm_param_grid = {'learning_rate': [.01, .015, .025, .05, .1],
#                   'Gamma': [.05, .1, .3, .5, .7, .9, 1],
                  'max_depth': [3, 5, 7, 9, 12, 15, 17, 25],
                  'min_child_weight': [1, 3, 5, 7],
                  'subsample': np.linspace(0.6, 1, 5)}

fit_params = {"early_stopping_rounds" : 20,
             "eval_metric" : "multi_error",
             "eval_set" : [(X_train, y_train), (X_val, y_val)]}

# Create a random search object
lgbm_random = RandomizedSearchCV(estimator = lgbm_clf,
                                param_distributions = lgbm_param_grid,
                                n_iter = 20, # n_iters in param combinations
                                scoring='accuracy',
                                n_jobs=-1,
                                cv = 5,
                                refit=True,
                                return_train_score = True,
                                verbose=10)

# Fit to the training data
lgbm_random.fit(X_train, y_train, **fit_params)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   39.6s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   51.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   59.1s
[Parallel(n_jobs=-1)]: Done  48 out of 100 | elapsed:  1.6min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  59 out of 100 | elapsed:  1.7min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  70 out of 100 | elapsed:  1.9min remaining:   47.8s
[Parallel(n_jobs=-1)]: Done  81 out of 100 | elapsed:  2.3min remaining:   32.6s
[Parallel(n_jobs=-1)]: Done  92 out of 100 | elapsed:  2.4min remaining:   12.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.5min finished


[1]	training's multi_error: 0.443536	training's multi_logloss: 1.97441	valid_1's multi_error: 0.468635	valid_1's multi_logloss: 2.00179
Training until validation scores don't improve for 20 rounds
[2]	training's multi_error: 0.2845	training's multi_logloss: 1.77681	valid_1's multi_error: 0.334563	valid_1's multi_logloss: 1.82763
[3]	training's multi_error: 0.226292	training's multi_logloss: 1.62152	valid_1's multi_error: 0.290898	valid_1's multi_logloss: 1.692
[4]	training's multi_error: 0.194136	training's multi_logloss: 1.4939	valid_1's multi_error: 0.259533	valid_1's multi_logloss: 1.58212
[5]	training's multi_error: 0.179965	training's multi_logloss: 1.3851	valid_1's multi_error: 0.252768	valid_1's multi_logloss: 1.48936
[6]	training's multi_error: 0.168847	training's multi_logloss: 1.29164	valid_1's multi_error: 0.240467	valid_1's multi_logloss: 1.40963
[7]	training's multi_error: 0.159908	training's multi_logloss: 1.20844	valid_1's multi_error: 0.234317	valid_1's multi_logloss: 1

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.015, 0.025,
                                                          0.05, 0.1],
                                        'max_depth': [3, 5, 7, 9, 12, 15, 17,
                                                      25],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ])},
                   return_train_score=True, scoring='accuracy', verbose=10)

In [48]:
pred_train = lgbm_random.best_estimator_.predict(X_train)
pred_val = lgbm_random.best_estimator_.predict(X_val)
pred_test = lgbm_random.best_estimator_.predict(X_test)

In [49]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred_test))

0.9923697405711794
0.8148831488314883
0.16483516483516483


In [50]:
lgbm_random.best_estimator_

LGBMClassifier(max_depth=9, min_child_weight=3, subsample=0.6)

In [51]:
cv_result_df = pd.DataFrame(lgbm_random.cv_results_)

df_list = []
for i in range(20):
    df_list.append(pd.DataFrame([cv_result_df.loc[i, "params"]]))
    
param_table = pd.concat(df_list)

param_table['mean_test_score'] = cv_result_df['mean_test_score'].values

param_table.sort_values(by='mean_test_score', axis=0)

,subsample,min_child_weight,max_depth,learning_rate,mean_test_score
0,1.0,7,3,0.010,0.690103
0,0.9,7,3,0.025,0.738936
0,0.6,7,5,0.015,0.758339
0,0.6,7,7,0.010,0.763244
0,1.0,5,9,0.010,0.766405
0,1.0,7,25,0.010,0.770111
0,0.7,7,12,0.010,0.770220
0,0.6,1,12,0.010,0.771419
0,0.6,1,9,0.010,0.772292
0,1.0,3,25,0.010,0.772401
